In [1]:
from pyspark import SparkContext, SparkConf

import psutil
import platform
from datetime import datetime

In [29]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

# let's print CPU information
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

# Disk Information
print("="*40, "Disk Information", "="*40)
print("Partitions and Usage:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    print(f"  File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_size(partition_usage.total)}")
    print(f"  Used: {get_size(partition_usage.used)}")
    print(f"  Free: {get_size(partition_usage.free)}")
    print(f"  Percentage: {partition_usage.percent}%")
# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

# Network information
print("="*40, "Network Information", "="*40)
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

======================================== System Information ========================================
System: Windows
Node Name: LAPTOP-N6L95A43
Release: 10
Version: 10.0.22000
Machine: AMD64
Processor: Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 3302.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 3302.00Mhz
CPU Usage Per Core:
Core 0: 7.5%
Core 1: 1.5%
Core 2: 6.2%
Core 3: 0.0%
Core 4: 3.1%
Core 5: 0.0%
Core 6: 1.5%
Core 7: 1.5%
Total CPU Usage: 5.6%
======================================== Memory Information ========================================
Total: 39.69GB
Available: 26.07GB
Used: 13.62GB
Percentage: 34.3%
==================== SWAP ====================
Total: 45.44GB
Free: 28.96GB
Used: 16.48GB
Percentage: 36.3%
======================================== Disk Information ========================================
Partitions and Usage:
=

In [32]:
# GPU information
import GPUtil
from tabulate import tabulate
print("="*40, "GPU Details", "="*40)
gpus = GPUtil.getGPUs()
list_gpus = []
for gpu in gpus:
    # get the GPU id
    gpu_id = gpu.id
    # name of GPU
    gpu_name = gpu.name
    # get % percentage of GPU usage of that GPU
    gpu_load = f"{gpu.load*100}%"
    # get free memory in MB format
    gpu_free_memory = f"{gpu.memoryFree}MB"
    # get used memory
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    # get total memory
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    # get GPU temperature in Celsius
    gpu_temperature = f"{gpu.temperature} °C"
    gpu_uuid = gpu.uuid
    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory, gpu_temperature, gpu_uuid
    ))

print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))

======================================== GPU Details ========================================
  id  name                                load    free memory    used memory    total memory    temperature    uuid
----  ----------------------------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  NVIDIA GeForce RTX 3060 Laptop GPU  20.0%   5604.0MB       540.0MB        6144.0MB        44.0 °C        GPU-35957a02-16fb-393e-286e-b75c123727cc


In [11]:
conf = SparkConf().setAppName("tmp").setMaster("local")
sc = SparkContext(conf=conf)

In [12]:
conf

In [13]:
sc

<SparkContext master=local appName=tmp>

In [65]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

distData.collect()

[1, 2, 3, 4, 5]

In [73]:
import pandas as pd 

distFile = pd.read_csv("Datos/trainsched.txt")

distFile

,train_id,station,time,diff_min
0,217,Gilroy,6:06a,9.0
1,217,San Martin,6:15a,6.0
2,217,Morgan Hill,6:21a,15.0
3,217,Blossom Hill,6:36a,6.0
4,217,Capitol,6:42a,8.0
5,217,Tamien,6:50a,9.0
6,217,San Jose,6:59a,NaN
7,324,San Francisco,7:59a,4.0
8,324,22nd Street,8:03a,13.0
9,324,Millbrae,8:16a,8.0


In [76]:
rddFile = sc.textFile("Datos/trainsched.txt")
rddFile.collect()

['train_id,station,time,diff_min',
 '217,Gilroy,6:06a,9.0',
 '217,San Martin,6:15a,6.0',
 '217,Morgan Hill,6:21a,15.0',
 '217,Blossom Hill,6:36a,6.0',
 '217,Capitol,6:42a,8.0',
 '217,Tamien,6:50a,9.0',
 '217,San Jose,6:59a,null',
 '324,San Francisco,7:59a,4.0',
 '324,22nd Street,8:03a,13.0',
 '324,Millbrae,8:16a,8.0',
 '324,Hillsdale,8:24a,7.0',
 '324,Redwood City,8:31a,6.0',
 '324,Palo Alto,8:37a,28.0',
 '324,San Jose,9:05a,null']

In [1]:
from pyspark import SparkContext, SparkConf
import pandas as pd

conf = SparkConf().setAppName("tmp").setMaster("local")
sc = SparkContext(conf=conf)

distFile = pd.read_csv("Datos/trainsched.txt").fillna(0)

#Crear un RDD
rddFile = sc.textFile("Datos/trainsched.txt")

rdd = sc.parallelize(distFile.loc[:,"diff_min"])
rdd.collect()
rdd.count()

rddtwo = rdd.map(lambda x: x)
rddtwo.collect()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (LAPTOP-N6L95A43 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.s